In [ ]:
import os
import datetime
import json
import numpy as np
import pandas as pd
import pprint

from IPython.display import display, HTML
from pymongo import MongoClient

In [ ]:
# Connect to Mongo host & port

client = MongoClient(os.environ['MONGODB_NAME'], 27017)

In [ ]:
# Input the name of the database you'd like to connect to. Example 'sacred_demo' or 'pythia_experiment'

db_name = input()

In [ ]:
# Connect to db

db = client.get_database(db_name)

In [ ]:
# This is the date of the experiments you'd like to see. Right now it is set at any experiment run after this date

date = datetime.datetime(2016, 7, 28, 0, 0, 0, 0)

In [ ]:
# This mongo query displays the results and the arguments passed in to the model

cursor = db.default.runs.find({'start_time': {"$gt" : date}}, {'config': True, 'result': True, '_id': False})
count = 0
for doc in cursor:
        count += 1
        print("\n-----------------------------")
        print("RUN NUMBER: " + str(count))
        print("-----------------------------")
        for key in doc['config'].keys():
            if key == 'args':
                result = {}
                # Print data used for the experiment
                args = doc['config']['args']
                print("\nData: " + str(args[0]) + "\n")
                # The way the config is set up, the first index is the data used and the remaining ones are the args
                for d in args[1:len(args)]:
                    result.update(d)
                df = pd.DataFrame(result).fillna(" ")
                key_list = []
                # I combine the dictionaries to make a nicer display for the pandas dataframe
                for key in result.keys():
                    for key2 in (result[key].keys()):
                        key_list.append(key2)
                args_df = df.reindex(key_list)
                display(args_df)
            else:
                print("Running with a seed of: " + str(doc['config']['seed']) + "\n")
        print("Results are:")
        display(pd.DataFrame(doc['result']))

In [ ]:
# The same concept as earlier: display experiments after the specified date below

date_pythia = datetime.datetime(2016, 8, 16, 0, 0, 0, 0)

In [ ]:
# Enter the name of the db you would like to see

db_name = input()

In [ ]:
# Connect to db

db = client.get_database(db_name)

In [ ]:
# This is for the logic behind the visualization. If all elements within the lists are false, 
# then the algorithm was not used

LDA_list = ['LDA_APPEND', 'LDA_COS', 'LDA_DIFFERENCE', 'LDA_PRODUCT', 'LDA_TOPICS']
BOW_list = ['BOW_APPEND', 'BOW_PRODUCT', 'BOW_COS', 'BOW_DIFFERENCE', 'BOW_TFIDF']
W2V_list = ['W2V_APPEND', 'W2V_DIFFERENCE', 'W2V_COS', 'W2V_PRODUCT']
W2V_associated_list = ['W2V_SIZE', 'W2V_WINDOW', 'W2V_WORKERS', 'W2V_MIN_COUNT', 'W2V_PRETRAINED']
CNN_list = ['CNN_APPEND', 'CNN_COS', 'CNN_PRODUCT', 'CNN_DIFFERENCE']
ST_list = ['ST_APPEND', 'ST_COS', 'ST_PRODUCT', 'ST_DIFFERENCE']
SVM_list = ['SVM']
WORDONEHOT_list = ['WORDONEHOT']
XGB_list = ['XGB']
LOG_REG_list = ['LOG_REG']

In [ ]:
cursor = db.default.runs.find({'start_time': {"$gt" : date_pythia}}, {'config': True, 'result': True, '_id': False})
count = 0

bool_listLDA = []
bool_listCNN = []
bool_listST = []
bool_listW2V = []
bool_listBOW = []

bool_listSVM = []
bool_listXGB = []
bool_listLOG_REG = []
bool_listWORDONEHOT = []

for doc in cursor:
    count += 1
    print("-------------------------")
    print("RUN NUMBER: " + str(count))
    print("-------------------------")
    df = pd.DataFrame(doc['config'], index=['value']).T
    
    for item in CNN_list:
        bool_listCNN.append(df.get_value(item, 'value'))
    if not any(bool_listCNN) == True:
            df = df.drop(CNN_list)
            if 'CNN_CHAR_VOCAB' in df.index:
                df = df.drop(['CNN_CHAR_VOCAB'])
            if 'CNN_VOCAB_TYPE' in df.index:
                df = df.drop(['CNN_VOCAB_TYPE'])
    
    for item in LDA_list:
        bool_listLDA.append(df.get_value(item, 'value'))
    if not any(bool_listLDA) == True:
        df = df.drop(LDA_list)
    
    for item in ST_list:
        bool_listST.append(df.get_value(item, 'value'))
    if not any(bool_listST) == True:
        df = df.drop(ST_list)
    
    for item in W2V_list:
        bool_listW2V.append(df.get_value(item, 'value'))
    if not any(bool_listW2V) == True:
        df = df.drop(W2V_list)
        for item in W2V_associated_list:
            if item in df.index:
                df = df.drop([item])
                
    for item in BOW_list:
        bool_listBOW.append(df.get_value(item, 'value'))
    if not any(bool_listBOW) == True:
        df = df.drop(BOW_list)        
        
    for item in SVM_list:
        bool_listSVM.append(df.get_value(item, 'value'))
    if not any(bool_listSVM) == True:
        df = df.drop(['SVM', 'SVM_C', 'SVM_GAMMA', 'SVM_KERNEL'])
        
    for item in XGB_list:
        bool_listXGB.append(df.get_value(item, 'value'))
    if not any(bool_listXGB) == True:
        df = df.drop(['XGB', 'XGB_COLSAMPLEBYTREE', 'XGB_LEARNRATE', 'XGB_MAXDEPTH', 'XGB_MINCHILDWEIGHT'])
    
    for item in LOG_REG_list:
        bool_listLOG_REG.append(df.get_value(item, 'value'))
    if not any(bool_listLOG_REG) == True:
        df = df.drop(['LOG_REG', 'LOG_PENALTY', 'LOG_C', 'LOG_TOL'])
        
    for item in WORDONEHOT_list:
        bool_listWORDONEHOT.append(df.get_value(item, 'value'))
    if not any(bool_listWORDONEHOT) == True:
        df = df.drop(['WORDONEHOT', 'WORDONEHOT_VOCAB'])
        
    display(df)
    print("\nRESULTS ARE:\n")
    display(pd.DataFrame(doc['result']))

In [ ]:
cursor = db.default.runs.find({ "experiment.name": "pythia_gridsearch"})

for doc in cursor:
    print(doc)